In [1]:
import sys, unittest, time, datetime
import urllib.request, urllib.error, urllib.parse
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import InvalidArgumentException
from tqdm.notebook import tqdm

import pymongo
from pymongo.errors import DuplicateKeyError
from pymongo import MongoClient
import traceback
import pickle

In [2]:
client = MongoClient()
db = client.youtube
collection = db.popscience

In [5]:
with open('popscience_playlists.txt') as f:
    yuotube_playlist_id_list = f.read().splitlines()

In [6]:
yuotube_playlist_id_list

['PL2jwxGybEFiuQVQtrLPaH7GNB8ak29634',
 'PL2jwxGybEFity3pyl5SnPrgh7ERNCd4FD',
 'PL2jwxGybEFivRT-UOnvV0RAh1ykwwRrmc',
 'PL2jwxGybEFitFmGV0RdBNgbNBWFPtWRe0',
 'PLYI-NYBPZXmizTp-oucPyUJL0zSWkgADo',
 'PLYI-NYBPZXmjduCa-984TbdL_Tep3sH8G',
 'PLYI-NYBPZXmhcO7mj6ixBVy77tvfA8GFM',
 'PLYI-NYBPZXmhrm7Quz2x9tg8VsjXT3GQd',
 'PLYI-NYBPZXmhoQt0lOq7lil9qFxaFBAHY',
 'PLYI-NYBPZXmgieQfNXuu5JyinD3enTNct',
 'PLYI-NYBPZXmhGZFRoBnfimimeDRPYz5TT',
 'PLYI-NYBPZXmgIKFqU3d8kzy1pvtDZw52x',
 'PLYI-NYBPZXmglqnB0IHwtkrHH69EVvI-t']

# Скачивание информации по плейлистам без повторов

In [22]:
%%time

sleep = 3
URLError_counter = 0


for playlist_id in tqdm(yuotube_playlist_id_list):
    try:
        playlist = f"https://www.youtube.com/playlist?list={playlist_id}"
        
        # Получение информации о плейлисте
        driver = webdriver.Chrome('../chromedriver')
        driver.get(playlist)
        time.sleep(sleep)
        user_data = driver.find_elements_by_xpath('//*[@id="description"]')
        pl_description = user_data[0].text
        user_data = driver.find_elements_by_xpath("//*[contains(text(), 'views')]")
        pl_views = user_data[1].text
        digits = [s for s in pl_views.split()[0] if s.isdigit()]
        if len(digits) == 0:
            pl_views = 0
        else:
            pl_views = int("".join(digits))
        driver.close()
#         driver.quit()
        p = Playlist(playlist)
        pl_title = p.title
          
        for url in p.video_urls:
            for i in range(10):
                try:
                    jon_tutorial = collection.find_one({"origin_url": url})
                    if jon_tutorial is not None:
                        break
                        
                    info = {}
                    try:
                        yt = YouTube(url)
                    except VideoPrivate:
                        print(f"Private video {url}")
                        break
                    except VideoUnavailable:
                        print(f"Unavailable video {url}")
                        break
                    info["title"] = yt.title
                    info["playlist_title"] = pl_title
                    info["playlist_description"] = pl_description
                    info["playlist_views"] = pl_views
                    info["date"] = yt.publish_date
                    info["origin_url"] = url
                    info["views"] = yt.views
                    info["duation"] = yt.length
                    info["author"] = yt.author
                    info["tags"] = yt.keywords
                    info["about"] = yt.description
                    info["rating"] = yt.rating
                    info["platform"] = "YouTube"
                    info["theme"] = yt.thumbnail_url
                    if yt.captions.get("ru") is not None:
                        caption = yt.captions.get('ru')
                        info["language"] = "ru"
                    elif yt.captions.get("a.ru") is not None:
                        caption = yt.captions.get('a.ru')
                        info["language"] = "a.ru"
                    if caption:
                        info["captions_xml"] = caption.xml_captions
                        info["captions_str"] = caption.generate_srt_captions()
                    result = collection.insert_one(info)
                except DuplicateKeyError:
                    print("============ DuplicateKeyError ============")
#                     traceback.print_exc()
                    continue
                except URLError:
                    print("============ URLError ============")
                    traceback.print_exc()
                    URLError_counter += 1
                    if URLError_counter == 20:
                        URLError_counter = 0
                        break
                    time.sleep(sleep)
                    continue
                except KeyError:
                    print("============ KeyError ============")
                    break
                else:
                    break
    except KeyboardInterrupt:
        print("============ KeyboardInterrupt ============")
        break
    except:
        print("============ UnnounError ============")
        traceback.print_exc()
        continue

  0%|          | 0/13 [00:00<?, ?it/s]

============ UnnounError ============


Traceback (most recent call last):
  File "<timed exec>", line 24, in <module>
NameError: name 'Playlist' is not defined


============ UnnounError ============


Traceback (most recent call last):
  File "<timed exec>", line 24, in <module>
NameError: name 'Playlist' is not defined


============ UnnounError ============


Traceback (most recent call last):
  File "<timed exec>", line 24, in <module>
NameError: name 'Playlist' is not defined


============ UnnounError ============


Traceback (most recent call last):
  File "<timed exec>", line 24, in <module>
NameError: name 'Playlist' is not defined


============ KeyboardInterrupt ============
CPU times: user 107 ms, sys: 50.8 ms, total: 158 ms
Wall time: 27.2 s
